# Preparation
This block is for preparing libraries and modules which will be used in the project. New libraries may be added during the project.

In [10]:
# Uncomment the following lines to install the required packages
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install pycountry

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry

datasets_path = 'datasets/'

# Questions

## What are the main countries of origin of immigrants in Italy, and how have these trends changed over the past 5 years?

## Generate mashed-up data

In [11]:
def convert_to_iso3(code):
    if pd.isna(code):  # Check if the value is NaN
        return None  # Keep it as NaN
    if code == "UK":  # Manually correct "UK" to "GBR"
        return "GBR"
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except AttributeError:
        # print(f"Warning: Country code '{code}' not found in ISO 3166-1 alpha-2!")  # Debugging
        return code  # Keep the original if not found

In [12]:
# load datasets
d4 = pd.read_csv(f'{datasets_path}IstatData/Immigrants - citizenship (IT1,28_185_DF_DCIS_MIGRAZIONI_2,1.0).csv')
d5 = pd.read_csv(f'{datasets_path}IstatData/Type of residence permit and citizenship (IT1,29_348_DF_DCIS_PERMSOGG1_1,1.0).csv')
d15 = pd.read_csv(f'{datasets_path}OECD/OECD.ELS.IMD,DSD_MIG@DF_MIG,1.0+ITA..A.B11.csv')

Sex_dictionary = {
  1: "Male",
  2: "Female",
  9: "Total",
}

Permit_Type_dictionary = {
  "LONGT": "Long-term",
  "NLONGT": "Short-term",
  "TOTAL": "Total",
}

d4_filtered = d4[d4['AGE'] == 'TOTAL'][['CITIZENSHIP',
                                        'SEX',
                                        # 'AGE',
                                        'TIME_PERIOD',
                                        'OBS_VALUE']]
d4_filtered = d4_filtered.rename(columns={'CITIZENSHIP': 'Country_Code',
                                          'SEX': 'Sex',
                                        #   'AGE': 'Age_range',
                                          'TIME_PERIOD': 'Year',
                                          'OBS_VALUE': 'Value'})
d4_filtered = d4_filtered[d4_filtered['Country_Code'].str.len() == 2]
# Convert country codes to ISO 3166-1 alpha-3
d4_filtered['Country_Code'] = d4_filtered['Country_Code'].apply(convert_to_iso3)
# Add column dataset = d4
d4_filtered['Dataset'] = 'Immigrants - citizenship'
d4_filtered['Dataset_Code'] = 'D4'


d5_filtered = d5[['MOSTREL_CCITENSHIP',
                  'SEX', 'TYPE_RES_PERMIT',
                  'TIME_PERIOD',
                  'OBS_VALUE']]
d5_filtered = d5_filtered.rename(columns={'MOSTREL_CCITENSHIP': 'Country_Code',
                                          'SEX': 'Sex',
                                          'TYPE_RES_PERMIT': 'Permit_Type',
                                          'TIME_PERIOD': 'Year',
                                          'OBS_VALUE': 'Value'})
d5_filtered = d5_filtered[d5_filtered['Country_Code'].str.len() == 2]
# Convert country codes to ISO 3166-1 alpha-3
d5_filtered['Country_Code'] = d5_filtered['Country_Code'].apply(convert_to_iso3)
# Add column dataset = d5
d5_filtered['Dataset'] = 'Type of residence permit and citizenship'
d5_filtered['Dataset_Code'] = 'D5'


# print All d13 columns
d15_filtered = d15[['CITIZENSHIP', 'Citizenship', 'Sex', 'TIME_PERIOD', 'OBS_VALUE']]
d15_filtered = d15_filtered.rename(columns={'CITIZENSHIP': 'Country_Code',
                                          'Citizenship': 'Country_Name',
                                          'TIME_PERIOD': 'Year',
                                          'OBS_VALUE': 'Value'})

# Add country name to d4 and d5
d4_filtered = d4_filtered.merge(d15_filtered[['Country_Code', 'Country_Name']], on='Country_Code', how='left')
d5_filtered = d5_filtered.merge(d15_filtered[['Country_Code', 'Country_Name']], on='Country_Code', how='left')

# Join d4 and d5
combined = pd.concat([d4_filtered, d5_filtered], ignore_index=True)

# Change Sex from integer to string
combined['Sex'] = combined['Sex'].map(Sex_dictionary)

# Change Permit_Type to improve readability
combined['Permit_Type'] = combined['Permit_Type'].map(Permit_Type_dictionary)

# Order columns: [Year, Country_Code, Country_Name, Sex, Value,  Dataset_Code, Dataset]
combined = combined[['Year', 'Country_Code', 'Country_Name', 'Sex', 'Value','Permit_Type' , 'Dataset_Code', 'Dataset']]
combined = combined.drop_duplicates()

# order by: [Country_Code, ]
combined = combined.sort_values(['Country_Name', 'Dataset_Code', 'Year'])

# Save to CSV
combined.to_csv(f'{datasets_path}mashup/italy_immigration_trends_by_country_and_permit.csv', index=False)


For visualization purposes, a secondary dataset has been created in which country codes have been converted from ISO 3166-1 Alpha-3 to ISO 3166-1 Alpha-2.

In [13]:
def convert_to_iso2(code):
    if pd.isna(code):  # Check if the value is NaN
        return None  # Keep it as NaN
    if code == "GBR":  # Manually correct "UK" to "GBR"
        return "GB"
    try:
        return pycountry.countries.get(alpha_3=code).alpha_2
    except AttributeError:
        # print(f"Warning: Country code '{code}' not found in ISO 3166-1 alpha-2!")  # Debugging
        return code  # Keep the original if not found
    

mashed_up = pd.read_csv(f'{datasets_path}mashup/italy_immigration_trends_by_country_and_permit.csv')
mashed_up['Country_Code'] = mashed_up['Country_Code'].apply(convert_to_iso2)

mashed_up.to_csv(f'{datasets_path}mashup/italy_immigration_trends_by_country_and_permit_iso2.csv', index=False)



## What are the demographic profiles of immigrants in Italy (age, gender, education level)?

In [14]:
d6 = pd.read_csv(f'{datasets_path}NoiItalia2024/output_file.csv')
d10 = pd.read_csv(f'{datasets_path}Immigrants.Stat/DCCV_TAXOCCU1_12012025110922125.csv')
d11 = pd.read_csv(f'{datasets_path}Immigrants.Stat/DCCV_TAXDISOCCU1_12012025111208359.csv')
d8 = pd.read_csv(f'{datasets_path}Immigrants.Stat/DCCV_OCCUPATIT1_12012025110421591.csv')

d6_filtered = d6.rename(columns={'Age_Group': 'Age_class'})
d6_filtered['Territory'] = 'Italy'
d6_filtered['Gender'] = 'total'
d6_filtered = d6_filtered[['Territory', 'Age_class', 'Gender', 'Education_Level', 'Year', 'Foreign_Percentage', 'Italian_Percentage']]

d10_filtered = d10[['Territory', 'Age class', 'Gender', 'Highest level of education attained', 'TIME', 'Value']]
d10_filtered = d10_filtered.rename(columns={'TIME': 'Year', 'Highest level of education attained': 'Education_Level', 'Age class': 'Age_class', 'Value': 'Employed_percentage'})  
# remove " years" prefix from Age_class
d10_filtered['Age_class'] = d10_filtered['Age_class'].str.replace(' years', '')

d11_filtered = d11[['Territory', 'Age class', 'Highest level of education attained', 'TIME', 'Value']]
d11_filtered = d11_filtered.rename(columns={'TIME': 'Year', 'Highest level of education attained': 'Education_Level', 'Age class': 'Age_class', 'Value': 'Unemployed_percentage'})
d11_filtered.insert(d11_filtered.columns.get_loc('Age_class') + 1, 'Gender', 'total')
# remove " years" prefix from Age_class
d11_filtered['Age_class'] = d11_filtered['Age_class'].str.replace(' years', '')
# Kepp the rows which Age_class is "15-64"
d11_filtered = d11_filtered[d11_filtered['Age_class'] == '15-64']

d8_filtered = d8[['Territory', 'Gender', 'Age class', 'Full-time/Part-time', 'TIME', 'Value']]
d8_filtered = d8_filtered.rename(columns={'TIME': 'Year', 'Full-time/Part-time': 'Work_Type', 'Value': 'Value', 'Age class': 'Age_class'})
# remove " years" prefix from Age_class
d8_filtered['Age_class'] = d8_filtered['Age_class'].str.replace(' years', '')
# Multiply by 1000 to get the actual number of people
d8_filtered['Value'] = d8_filtered['Value'] * 1000
d8_filtered["Education_Level"] = 'total'

# Cast all year columns to string
d6_filtered["Year"] = d6_filtered["Year"].astype(str)
d10_filtered["Year"] = d10_filtered["Year"].astype(str)
d11_filtered["Year"] = d11_filtered["Year"].astype(str)
d8_filtered["Year"] = d8_filtered["Year"].astype(str)


In [15]:
# Mapping dictionary for the d6_filtered dataset
mapping_d6 = {
    'Licenza_Media': 'no educational degree, primary and lower secondary school certificate',
    'Diploma': 'upper and post secondary',
    'Titolo_Universitario': 'tertiary (university, doctoral and specialization courses)'
}

# Apply the mapping to d6_filtered while preserving any values not in the mapping (like 'total')
d6_filtered['Education_Level'] = d6_filtered['Education_Level'].apply(
    lambda x: mapping_d6.get(x, x)
)

In [16]:
# standardize the Year column in all datasets
# From d8, d10, d11 remove all the querterly data and just keep the yearly data
d8_filtered = d8_filtered[d8_filtered['Year'].str.contains('Q') == False]
d10_filtered = d10_filtered[d10_filtered['Year'].str.contains('Q') == False]
d11_filtered = d11_filtered[d11_filtered['Year'].str.contains('Q') == False]

# In all datasets remove years before 2019
d8_filtered = d8_filtered[d8_filtered['Year'].astype(int) >= 2019]
d10_filtered = d10_filtered[d10_filtered['Year'].astype(int) >= 2019]
d11_filtered = d11_filtered[d11_filtered['Year'].astype(int) >= 2019]
d6_filtered = d6_filtered[d6_filtered['Year'].astype(int) >= 2019]

In [17]:
# Combine D10 and D11
d10_without_age = d10_filtered.drop(columns=['Age_class'])
d10_without_age = d10_without_age.drop_duplicates()

d11_without_age = d11_filtered.drop(columns=['Age_class'])
d11_without_age = d11_without_age.drop_duplicates()

d8_without_age = d8_filtered.drop(columns=['Age_class'])
d8_without_age = d8_without_age.drop_duplicates()

d10_d11 = pd.merge(d11_without_age, d10_without_age, on=['Territory', "Education_Level", "Year", "Gender"], how='outer')

combined = pd.merge(d10_d11, d8_without_age, on=['Territory', "Year", "Gender", 'Education_Level'], how='outer')

combined['Age_class'] = 'Total'

# Combine with d6_filtered
combined = pd.merge(combined, d6_filtered, on=['Territory', "Year", "Gender", 'Education_Level', "Age_class"], how='outer')



In [18]:
# Define a custom sorting key function
def custom_sort_key(row):
    territory_priority = 0 if row['Territory'] == 'Italy' else 1
    year_priority = 0 if row['Year'] == 'total' else 1
    gender_priority = 0 if row['Gender'] == 'total' else 1
    education_priority = 0 if row['Education_Level'] == 'total' else 1
    work_type_priority = 0 if row['Work_Type'] == 'total' else 1
    
    return (territory_priority, row['Territory'], year_priority, row['Year'], gender_priority, row['Gender'], education_priority, row['Education_Level'], work_type_priority, row['Work_Type'])

# Sort the dataframe using the custom sorting key
combined['Territory_priority'] = combined['Territory'].apply(lambda x: 0 if x == 'Italy' else 1)
combined['Year_priority'] = combined['Year'].apply(lambda x: 0 if x == 'total' else 1)
combined['Gender_priority'] = combined['Gender'].apply(lambda x: 0 if x == 'total' else 1)
combined['Education_Level_priority'] = combined['Education_Level'].apply(lambda x: 0 if x == 'total' else 1)
combined['Work_Type_priority'] = combined['Work_Type'].apply(lambda x: 0 if x == 'total' else 1)

combined_sorted = combined.sort_values(by=['Territory_priority', 'Territory', 'Year_priority', 'Year', 'Gender_priority', 'Gender', 'Education_Level_priority', 'Education_Level', 'Work_Type_priority', 'Work_Type'])

# Drop the priority columns as they are no longer needed
combined_sorted = combined_sorted.drop(columns=['Territory_priority', 'Year_priority', 'Gender_priority', 'Education_Level_priority', 'Work_Type_priority'])

# Reset the index if needed
combined_sorted = combined_sorted.reset_index(drop=True)

combined_sorted.to_csv(f'{datasets_path}mashup/italy_employment_education_trends.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'datasets/mashup/italy_employment_education_trends.csv'